In [51]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn import metrics
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F
import torch
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import mat4py
import pandas as pd
import data_tracks
import models
from matplotlib_venn import venn3, venn2
from sklearn.model_selection import KFold
from colorama import Fore
import pickle as pkl
import viz
from style import *
import analyze_helper
import data_tracks
import train

outcome_def = 'y_consec_sig'
out_dir = 'results/outcome=y_consec_sig'
results = analyze_helper.load_results(out_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# compare all models

**look at prediction metrics**

In [ ]:
r = results
r = r[[k for k in r if not 'std' in k]]
r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
r = r.sort_values(by=['accuracy', 'balanced_accuracy'], ascending=False)
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])

**look at feat importances**

In [ ]:
r = results
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
r = r[r.accuracy > 0.65]
keys = [k for k in r if '_f' in k]
keys_remapped = {k: k.replace('_f', '') for k in keys}
r = r[keys].rename(columns=keys_remapped)
# r = r.sort_values('lifetime')
# r = r[r.index.str.contains('39')]
r = r[~r.index.str.contains('=11')]
# r = r[~r.index.str.contains('=7')]
r = r[~r.index.str.contains('=15')]
r = r.rename(columns={'mean_square_displacement': 'msd', 'total_displacement': 'td'})

# r = r[r.index.str.contains('11')]
# r = r[r2.index.str.contains('ros')]
# r = r[r.index.str.contains('none')]
r = r[[k for k in r if not 'std' in k]]


def rank(r):
    '''Rank feature importances appropriately
    '''
    r = r.abs()
    r = r.rank(axis=1, ascending=False, method='min')
    return r
# 
r = rank(r)
r = r.reindex(r.mean().sort_values(ascending=True).index, axis=1) # sort cols by mean rank

idxs = r.index
r.insert(0, 'acc', results.loc[idxs]['accuracy'])
r = r.sort_values('acc', ascending=False)

subset = list(r.keys())
subset.remove('acc')
r.fillna(0).style.background_gradient(cmap='viridis_r', axis=1, subset=subset) # rows differently colored

# analyze model predictions

In [2]:
df = data_tracks.get_data()
n = df.shape[0]

# normalize and store
X = df[data_tracks.get_feature_names(df)]
X_mean = X.mean()
X_std = X.std()
ks = list(X.keys())
norms = {ks[i]: {'mu': X_mean[i], 'std': X_std[i]} for i in range(len(ks))}
X = (X - X_mean) / X_std
y = df[outcome_def].values


# split testing data based on cell num
cv_idx = data_tracks.cell_nums_train[0]
idxs_test = df.cell_num.isin([cv_idx]) # this is the cv set for model's trained in the first cv fold
X_test, Y_test = X[idxs_test], y[idxs_test]

# look at best models
r = results
r = r.sort_values('accuracy', ascending=False)
idx = np.array(r.index)
print(idx[:10])

['mlp2_15_ros_select_rf=3' 'svm_25_ros_select_rf=11'
 'svm_25_ros_select_rf=5' 'mlp2_45_ros_select_rf=7'
 'svm_15_ros_select_rf=5' 'mlp2_25_ros_select_rf=7'
 'mlp2_25_ros_select_rf=3' 'svm_25_ros_select_rf=3'
 'svm_45_ros_select_rf=5' 'svm_15_ros_select_rf=11']


### look at single model

In [3]:
model_name = idx[0]

In [52]:
d_full_cv, idxs_cv = analyze_helper.get_data_over_folds(model_name, out_dir, df.cell_num, X, y)
y_full_cv = df[outcome_def].iloc[idxs_cv].values
preds = d_full_cv[model_name].values
preds_proba = d_full_cv[model_name + '_proba'].values

results_individual = pkl.load(open(oj(out_dir, f'{model_name}.pkl'), 'rb'))
assert np.mean(preds==y_full_cv) == np.average(results_individual['cv']['accuracy'], weights=results_individual['num_pts_by_fold_cv']), 'did not properly load model/data'

In [ ]:
viz.viz_errs_lifetime(X.iloc[idxs_cv], preds, preds_proba, y_full_cv, norms)

In [ ]:
viz.viz_errs_spatially(df, idxs_cv, preds, y_full_cv)

In [ ]:
viz.viz_errs_outliers(X.iloc[idxs_cv], preds, y_full_cv, num_feats_reduced=20)

In [ ]:
idxs_fp = preds > y_full_cv
idxs_fn = preds < y_full_cv
print('fp')
viz.viz_biggest_errs(df['X'].iloc[idxs_cv][idxs_fp], y_full_cv[idxs_fp], preds[idxs_fp], preds_proba[idxs_fp])    
print('fn')
viz.viz_biggest_errs(df['X'].iloc[idxs_cv][idxs_fn], y_full_cv[idxs_fn], preds[idxs_fn], preds_proba[idxs_fn])    

### look at many models

In [72]:
# model_names = ['mlp2_11_none', 'svm_35_none', 'logistic_4_none', 'rf_9_none']
# model_names = ['mlp2_11_none', 'mlp2_9_none', 'mlp2_4_none']
model_names = idx[:3]
d_full_cv, idxs_cv = analyze_helper.get_data_over_folds(model_names, out_dir, df.cell_num, X, y)
y_full_cv = df[outcome_def].iloc[idxs_cv].values

mlp2_15_ros_select_rf=3
svm_25_ros_select_rf=11
svm_25_ros_select_rf=5


**venn-diagram**

In [ ]:
sets = []
for model_name in model_names:
    args = np.argwhere(d_full_cv[model_name] != y_full_cv)
    sets.append(set(args.flatten().tolist()))
    
plt.figure(dpi=300)
plt.title('venn diagram of shared errors')
venn3(sets, model_names[:3])
plt.show()

**ensemble err**

In [89]:
d_full_cv, y_full_cv = analyze_helper.get_data_over_folds(model_names[0:3], out_dir, df.cell_num, X, y)

# ensemble
d_full_cv_probs = d_full_cv[[k for k in d_full_cv.keys() if 'proba' in k]]
preds_soft = d_full_cv_probs.sum(axis=1) / d_full_cv_probs.shape[1]
print(preds_soft)

for score_name in train.scorers.keys():
    print(score_name, train.scorers[score_name](y_full_cv, preds_soft > 0.5))
# metrics.accuracy_score(y_ensemble, preds_soft>0.5) 

0       0.070164
1       0.840049
2       0.088628
3       0.120895
4       0.905893
5       0.129751
6       0.510093
7       0.140856
8       0.114931
9       0.517429
10      0.124169
11      0.125611
12      0.922058
13      0.074506
14      0.068827
15      0.040415
16      0.860703
17      0.909912
18      0.541398
19      0.123836
20      0.851159
21      0.147195
22      0.896946
23      0.134209
24      0.137617
25      0.136420
26      0.155908
27      0.861863
28      0.893473
29      0.895077
          ...   
1003    0.066882
1004    0.052670
1005    0.565268
1006    0.112738
1007    0.910013
1008    0.164609
1009    0.198462
1010    0.055731
1011    0.105055
1012    0.876831
1013    0.168407
1014    0.125791
1015    0.189192
1016    0.906719
1017    0.875765
1018    0.918448
1019    0.914523
1020    0.098564
1021    0.894506
1022    0.118730
1023    0.149869
1024    0.098325
1025    0.079775
1026    0.067586
1027    0.071098
1028    0.891601
1029    0.913778
1030    0.0689

/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1855: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].